## **Four Square APIs**

This tutorial provides a brief description of how getting started with Foursquare API. You will also find a good tutorial here:
https://www.kaggle.com/code/kristoft/tutorial-foursquare-api-search


**Getting Started**

Foursquare API provides a range of tools for developers to incorporate the up-to-date location data to enhance their projects.

To start using Foursquare, first, let’s create a new account at https://developer.foursquare.com/ . The default account is called Sandbox Account. Sandbox Account provides you a limited set of tools, for instance, you‘ll be able to make only 950 Regular Calls / Day, and get 1 Photo and 1 Tip per Venue. Not too much, but it should be enough for a “toy” dataset (sure, you may also upgrade your account for future projects). Foursquare provides a simple UI to manage your account and to check your daily stats (like how many calls did you make). Once the account is created, you will be assigned credentials: the Client ID and the Client Secret.

Ok, all set up and we can jump into coding. First and foremost, let’s import some necessary libraries: pandas.io.json(to tranforming json file into a pandas dataframe library), folium (plotting library), geopy.geocoders (module to convert an address into latitude and longitude values), and requests (library to handle requests).

In [3]:
!pip install geopy
from pandas import json_normalize
import folium
import pandas as pd
from geopy.geocoders import Nominatim
import requests

  Obtaining dependency information for geopy from https://files.pythonhosted.org/packages/e5/15/cf2a69ade4b194aa524ac75112d5caac37414b20a3a03e6865dfe0bd1539/geopy-2.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for geographiclib<3,>=1.52 from https://files.pythonhosted.org/packages/9f/5a/a26132406f1f40cf51ea349a5f11b0a46cec02a2031ff82e391c2537247a/geographiclib-2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 3.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.3 MB/s eta 0:00:00


Next, lets define our Foursquare credentials and version into python code:

In [4]:
CLIENT_ID = "AX1O0GXCKC2YOCKBYWQRXPXQFLZBAZE5PKHGOPA5QRMJQWKI" # your Foursquare ID
CLIENT_SECRET = "XH34Z0ZQXNJSOWAL5EP4000O0UBYMJVI1UEFJTPKO0SZF24G" # your Foursquare Secret
VERSION = "20180604"
LIMIT = 100

Now, let’s assume you just moved to a new place which called Upenn, and you want to explore your neighbourhood. So, let’s define a query to search the top 100 venues that are within 5000 meters from UPenn.

In [5]:
# Select first building (which in our case is Penn):
neighborhood_name = 'Penn University'
neighborhood_latitude = 39.952
neighborhood_longitude = -75.193

# limit of number of venues returned by Foursquare API
LIMIT = 100
radius = 2500

And now we define the corresponding URL and send the GET Request to Foursquare API and examine the results.

In [6]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    LIMIT)
results = requests.get(url).json()
print(results)

{'meta': {'code': 200, 'requestId': '671a6c1948953d5437b530b0'}, 'response': {'queryRefinements': {'target': {'type': 'path', 'url': '/venue/explore', 'params': {'ll': '39.952000,-75.193000', 'radius': '2500'}}, 'refinements': [{'query': 'Food'}, {'query': 'Nightlife'}, {'query': 'Coffee'}, {'query': 'Shops'}, {'query': 'Arts'}, {'query': 'Outdoors'}]}, 'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': '$-$$$$', 'key': 'price'}, {'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'Philadelphia', 'headerFullLocation': 'Philadelphia', 'headerLocationGranularity': 'city', 'totalResults': 208, 'suggestedBounds': {'ne': {'lat': 39.974500022500024, 'lng': -75.16370373082528}, 'sw': {'lat': 39.92949997749997, 'lng': -75.22229626917472}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '40b28c8

Next, get relevant part of JSON and transform it into a pandas dataframe:

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['groups'][0]['items']

# tranform venues into a pandas dataframe
dataframe = json_normalize(venues)
dataframe.head()


,tips,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.contact.phone,venue.contact.formattedPhone,venue.contact.twitter,venue.contact.facebook,...,photo.suffix,photo.width,photo.height,photo.visibility,venue.location.crossStreet,venue.venuePage.id,venue.storeId,venue.venueRatingBlacklisted,venue.contact.instagram,venue.location.neighborhood
0,"[{'id': '4e8f58cc0cd6d5e025a5aea2', 'createdAt...",e-0-40b28c80f964a520f7f81ee3-0,0,"[{'summary': 'This spot is popular', 'type': '...",40b28c80f964a520f7f81ee3,White Dog Cafe,2153869224,(215) 386-9224,whitedogphilly,98469539146,...,/3134736_cWwz6c0bFVnnd3_chhnxC0a17EBJZYuXxWlYa...,1440,1920,public,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'id': '531dbf1a11d274d268865546', 'createdAt...",e-0-531740af498e0b817696bbb1-1,0,"[{'summary': 'This spot is popular', 'type': '...",531740af498e0b817696bbb1,Federal Donuts,2672758489,(267) 275-8489,federaldonuts,NaN,...,/3781018_VNWlIBJS0b4UEx4tUa_Gdt4i3mnHKgEO-39bP...,720,960,public,at 34th St,NaN,NaN,NaN,NaN,NaN
2,"[{'id': '5d331f9d429bfc00256ac685', 'createdAt...",e-0-584601eb03e29a3ad375b50b-2,0,"[{'summary': 'This spot is popular', 'type': '...",584601eb03e29a3ad375b50b,The Study at University City,2153871400,(215) 387-1400,NaN,NaN,...,/523630644_LbRWxwfZO8jIORuQ7OoW_cbomYhy4GNoadm...,1080,1920,public,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'id': '4c8f9a71c86595213c360929', 'createdAt...",e-0-3fd66200f964a52009ee1ee3-3,0,"[{'summary': 'This spot is popular', 'type': '...",3fd66200f964a52009ee1ee3,Pod,2153871803,(215) 387-1803,StarrRestaurant,81183216293,...,/6531240_U29i-VVz-tOGOIHzgHHXpbD46eV0Y6afKgpiP...,720,960,public,btw 36th & 37th,32706361,NaN,NaN,NaN,NaN
4,"[{'id': '5b19bca7121384002cc412f7', 'createdAt...",e-0-5aae8fdcdee770732ce8972a-4,0,"[{'summary': 'This spot is popular', 'type': '...",5aae8fdcdee770732ce8972a,Goldie,2672390777,(267) 239-0777,NaN,NaN,...,/528180147_KfJdalnq7_zz0mMPit9ZpSVeREHe_h6IOgU...,975,1920,public,NaN,NaN,NaN,NaN,NaN,NaN



Define information of interest and filter the dataframe

In [8]:
# filter columns
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =dataframe.loc[:, filtered_columns]# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head(10)

,name,categories,lat,lng
0,White Dog Cafe,American Restaurant,39.953548,-75.192949
1,Federal Donuts,Donut Shop,39.953671,-75.193126
2,The Study at University City,Hotel,39.954434,-75.190067
3,Pod,Asian Restaurant,39.953971,-75.196037
4,Goldie,Falafel Restaurant,39.953156,-75.192737
5,University of Pennsylvania Museum of Archaeolo...,History Museum,39.949507,-75.191684
6,World Cafe Live,Music Venue,39.952214,-75.185151
7,Avril 50,Coffee Shop,39.953538,-75.192546
8,Penn Park,Park,39.951241,-75.185166
9,Louie Louie,Restaurant,39.953511,-75.196071


Finally, lets visualize the venues around Penn. For this purpose I use Folium library (Folium is completely free). Generating the map is straigtforward in Folium. We only create a Folium Map object and display it. So, I generate a map centred around the Penn campus. Bellow is the python code:

In [10]:
venues_map = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=15)

folium.features.CircleMarker(
    [neighborhood_latitude, neighborhood_longitude],
    radius=10,
    color='red',
    popup='UPENN',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)


# add all venues as blue circle markers
for lat, lng, label in zip(nearby_venues.lat, nearby_venues.lng, nearby_venues.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map